# Presentation Skeleton

* Goal was to see if crime rates differed between gentrified and non-gentrified neighborhoods over time, to see both if changes in crime correlate with gentrification, later we might use the analysis to compare with another city. 

* City of Chicago Data called by API (see below), looks at three years [2002, 2010, 2018], and makes a call for up to 400,000 reported cries per year. Because the API throttles API calls that are greater than 400,000 records long, the code below is broken up such that the operations for each year take place one at a time. More elegant code could abbreviate the process, but slowing down the processing speed made the API call more stable and consistent. 

* To connect crime data, with fbi code books, gentrification indeces and demographic information, the majority of the code is a series of pivot-like data manipulations, resulting in tables that provide summary counts by crime category [violent, nonviolent, white collar] and in neighborhoods that were coded [gentrified, not gentrified]. From an intial 1.2m reported crimes, our subest is closer to 300,000, including 30 of 70 community areas.

* Next we merge with demographic data to adjust count of reported crimes by population to generate a crime rate. The crime rate per neighborhood type, crime type, and year were then plotted in a series of box plots. 

#### Example Socrata Module for City of Chicago Data. https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi
* I don't use the sodapy package, the code is more efficient, but I'm trying to demonstrate that I'm learning the DS_Bootcamp code

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

# import pandas as pd
from sodapy import Socrata

MyAppToken = 'PoLZLhceSUgObx1Disd6WIQS1'

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 MyAppToken,
                 username="stephen.monteiro@northwestern.edu",
                 password="FackeTestPw0rd")

In [2]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pprint as pp
import os
import csv
year = [2002, 2010, 2018]


In [3]:
# fbi_codex Needed to index Crime Reports into summary categories
fbi_codex = pd.read_csv('crime_resources/fbi_codex.csv')
fbi_index = fbi_codex[['fbi_Crime_type','fbi_code_key','ds_fbi_group']]
fbi_index = fbi_index.dropna()

# gent_codex Needed to index Crime Reports into summary categories
gent_codex = pd.read_csv('../Resources/UIC_gentrification_index_neighborhood_list_final.csv')
gent_concat = gent_codex["Type"].str.split(" - ", n = 1, expand = True) 
gent_codex["Classification"] = gent_concat[1]
gent_codex["Class_Code"] = gent_concat[0]
gent_fin = gent_codex.rename(columns={"No.": "community_area"})

# cca_codex will allow us to convert raw crime into a crime rate for 2010 and an approzimation for 2002
cca_codex = pd.read_csv('../Resources/ReferenceCCA20102014.csv')

# cca_codex2018 will allow us to convert raw crime into a crime rate for 2018

cca_codex2018 = pd.read_csv('../Dev/Demographics/Resources/DemoData2018.csv')

# display all imported csvs so we can remember what we're working with later
display(fbi_index.head())
display(gent_fin.head())
display(cca_codex.head())
display(cca_codex2018.head())

FileNotFoundError: File b'crime_resources/fbi_codex.csv' does not exist